In [47]:
import os

In [48]:
os.chdir(r"C:\Users\Nazbeen-Ai\Documents\My Projects\Machine Learning Projects\End To End Chickens-desease-Classification")

In [49]:
pwd

'C:\\Users\\Nazbeen-Ai\\Documents\\My Projects\\Machine Learning Projects\\End To End Chickens-desease-Classification'

In [50]:
from dataclasses import  dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [51]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [52]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [53]:
import os
from  urllib import request
from zipfile import ZipFile
from cnnClassifier import  logger
from cnnClassifier.utils.common import get_size
import requests

In [54]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]
    
    
    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    

    
    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

        

In [55]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e: 
    raise e

[2024-03-24 07:37:21,382: INFO: yaml file: config\config.yaml loaded successfully]
[2024-03-24 07:37:21,395: INFO: yaml file: params.yaml loaded successfully]
[2024-03-24 07:37:21,397: INFO: Creating Directory; artifacts]
[2024-03-24 07:37:21,397: INFO: Creating Directory; artifacts/data_ingestion]


BadZipFile: File is not a zip file